In [40]:
#MODULOS DE SELENIUM
from selenium import webdriver
from selenium.webdriver.common.by import By

#MODULOS SECUNDARIOS
import pandas as pd
import numpy as np
import os
import re
import time
import ssl
import sys
import math
import traceback

#MODULO ALERTAS DESACTIVADAS
import warnings
warnings.filterwarnings('ignore')

#MODULO PROPIOS
from funciones_primarias import validacion
from funciones_secundarias import barra_carga,scroll_down

In [41]:
'''Protocoloes SSL'''
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [42]:

'''
OBJETO DRIVER 
    - Opciones 
    - URL
'''

#Opciones Firefox
firefox_profile = webdriver.FirefoxOptions()
firefox_profile.set_preference("browser.download.folderList", 2)
firefox_profile.set_preference("browser.download.dir", 'descargas')
firefox_profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")

# Navegador Firefox con el perfil configurado
driver = webdriver.Firefox(options=firefox_profile)

#Instancia driver -  Acceso a la URL
# PAGINA_URL = 'https://www.youtube.com/watch?v=lsId3AoZhbc&t=1997s'
PAGINA_URL = 'https://www.youtube.com/watch?v=pgg7ciV1Vfk&t=2277s'

driver.get(PAGINA_URL)

In [43]:
''' 
VALIDACION 

Funcionamiento de acceso a la URL, navegacion y captura de segmentos del en X.Path
Prueba del Scroll y de la Estrategia de Espera para la carga de mensajes

'''

#ubicar el primer comentario que se encuentra en la pagina, es decir el primer
#contenedor dentro de contenedor macro, esto para tener el punto de referencia de :
#donde se encuentra el bloque de comentarios y donde inicia

seccion_comments = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-comments/ytd-item-section-renderer/div[3]/ytd-comment-thread-renderer['

# scroll_down(5,1,1080,driver)
x=0
driver.execute_script('scrollBy(0,350)')
time.sleep(5)
barra_carga(20)
validacion(driver,seccion_comments)

1234567891011121314151617181920-------------]
SeccionComentario Encontrada
UserName Encontrado


In [46]:
#VARIABLES PARA EL SCRAPING
comentarios_recorridos = list()
centinela = True
comment_externo = 1
comment_interno = 1
base_final = pd.DataFrame()


while centinela == True:
    try:
        
        try:
            driver.find_element(By.XPATH, value= '/html/body/div[7]/div/div[1]/div[2]/div')
            print('Hay captcha...')
            time.sleep(30)
        except:
            pass

        # Level 1 comment principal
        level = 1
        # Detectar el DISPLAY NAME del usuario
        try:
            display_name = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/ytd-comment-renderer/div[3]/div[2]/div[1]/div[2]/h3/a/span").text
            print(display_name)
        except:
            centinela = False
            break

        try:    
        # Detectar NOMBRE DE USUARIO (toma como punto de referencia en Display Name del bloque anterior)
            username_thread = driver.find_element(by=By.PARTIAL_LINK_TEXT,value=display_name).get_attribute("href")
        # Detectar comment principal
            comment = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/ytd-comment-renderer/div[3]/div[2]/div[2]/ytd-expander/div/yt-formatted-string").text
    #         # Detectar fecha
    #         fecha = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/div[1]/div[1]/p[2]/span[1]").text
    #         # Detectar número de likes
    #         likes = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/div[1]/div[1]/p[2]/div/span").text
            print(str(comment))

        except:
            username_thread = driver.find_element(by=By.PARTIAL_LINK_TEXT,value=display_name).get_attribute("href")
            # Detectar comment principal
            comment = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/ytd-comment-renderer/div[3]/div[2]/div[2]/ytd-expander/div/yt-formatted-string").text
            print( 'Extraccion dos'+ str(comment))    

        # Creacion de Dataframe con los datos de cada comentario, y la base previa 
        d = {"level": level, "display_name": display_name, "username":username_thread, "comment":comment}
        base_temporal = pd.DataFrame(data=d,index=[comment_externo])
        base_final = pd.concat([base_final, base_temporal], ignore_index=True)
    
        # Actualizacion del loop para continuar con el recorrido
        comentarios_recorridos.append(comment_externo)
        comment_externo = comment_externo + 1
        print("Acabé el comment " + str(comment_externo - 1) + ' perteneciente al usuario: ' + str(display_name))
        
        x=0
        driver.execute_script('scrollBy(0,100)')
        time.sleep(5)
        
    except Exception as e:
        traceback.print_exc()
        time.sleep(50)
        scroll_down(3, 1, 850,driver)
        # with open(r'C:\Users\JOSE\Desktop\Trabajo\BX\Tiktok\progreso.txt', 'w') as fp:
        #     fp.write(str(comentarios_recorridos[-1]))
        print(f'Error {type(e)}: e')
        # base_final.to_excel('Base_tiktok_medicossinfronteras.xlsx')
        
base_final.to_excel('base_youtube_cuchicheo.xlsx')   

@janetharboleda5797
Me encanta este espacio que empodera notablemente a nuestra gente. 
Acabé el comment 1 perteneciente al usuario: @janetharboleda5797
@LaMonaSoyYo
Gracias por estos espacios tan importantes para deconstrucción 
Acabé el comment 2 perteneciente al usuario: @LaMonaSoyYo
@mariaisabeljaramillo8772
Me encanta este programa....para mí la preparación es muy importante !!! ..y es muy distinto  al que no tiene. Preparación   súper programa
Acabé el comment 3 perteneciente al usuario: @mariaisabeljaramillo8772
@yocreoencorinto
El conocimiento es poder, por eso debemos seguirnos preparando para transformar nuestros territorios.
Acabé el comment 4 perteneciente al usuario: @yocreoencorinto
@hugosalazarpelaez5913
Buen programa interesante diálogo, ojalá más espacios como este.
Acabé el comment 5 perteneciente al usuario: @hugosalazarpelaez5913
@marcelasalazar305
Qué buena charla!! 
Acabé el comment 6 perteneciente al usuario: @marcelasalazar305
@user-xi4bi6vm5f
Brutal esta entrev